In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

###### X_MLP/W10_L6(last lec.)-dip-iitm: [AdaBoost and GradientBoost Regressor on California Housing](https://www.youtube.com/watch?v=yJjCDkjNNaM&list=PLyGVjd4KQp13HB4vo0f_ztrpR8dPrDNKP&index=76)

# AdaBoost and GradientBoost Regressor on California Housing Dataset

In this notebook, we will apply ensemble tecniques regression problem in california housing datasets.

We will make use of 
- AdaBoost regressor
- Gradient boosting regressor
- XGBoost regressor

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.ensemble import AdaBoostRegressor,\
    GradientBoostingRegressor

from sklearn.metrics import mean_absolute_error

from sklearn.model_selection import cross_validate,\
    train_test_split, RandomizedSearchCV, ShuffleSplit

In [ ]:
np.random.seed(306)

Lets use`ShuffleSplit` as cv with 10 splits and 20% examples set aside as test examples.

In [ ]:
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)

Let's download the data and split it into training and test sets.

In [ ]:
features, labels = fetch_california_housing(as_frame=True, return_X_y=True)
labels*=100

# train_test split
com_train_features, test_features, com_train_labels, test_labels = \
    train_test_split(features, labels, random_state=42)


train_features, dev_features, train_labels, dev_labels =\
    train_test_split(com_train_features, com_train_labels, random_state=42)

## Training different regressors

In [ ]:
def train_regressor(estimator, X_train, y_train, cv, name):
    cv_results = cross_validate(estimator, 
                               X_train, 
                               y_train,
                               cv=cv,
                               scoring="neg_mean_absolute_error",
                               return_train_score=True,
                               return_estimator=True)
    
    cv_train_error = -1 * cv_results['train_score']
    cv_test_error = -1 * cv_results['test_score']
    
    print(f"On an average, {name} makes an error of "
          f"{cv_train_error.mean():.3f}k +/- {cv_train_error.std():.3f}k on the training set.")
    print(f"On an average, {name} makes an error of"
          f"{cv_test_error.mean():.3f}k +/- {cv_test_error.std():.3f}k on teh test set.")

In [ ]:
#@title AdaBoostRegressor
train_regressor(
    AdaBoostRegressor(), com_train_features,
    com_train_labels, cv, 'AdaBoostRegressor')

In [ ]:
#@title GradientBoostingRegressor
train_regressor(
    GradientBoostingRegressor(), com_train_features,
    com_train_labels, cv, 'GradientBoostingRegressor')

### XGBoost

In [ ]:
!pip install Xgboost

Extreme gradient boosting(XGBoost) is the latest boosting technique. It is more regularized form of gradient boosting. With regularization, it is able to achieve better generalization performance than gradient boosting.

In [ ]:
from xgboost import XGBRegressor
xgb_regressor = XGBRegressor(objective='reg:squarederror')

In [ ]:
train_regressor(
    xgb_regressor, com_train_features,
    com_train_labels, cv, 'XGBoostRegressor')

In [ ]:
nan